In [1]:
import pandas as pd 
import numpy as np
import re 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pickle , joblib
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

ModuleNotFoundError: No module named 'pandas'

In [3]:
x , y = [],[]
with open("SMSSpamCollection.txt","r") as file:
    data = file.read()
    #iterate through each line
    for data_item in data.split("\n"):
#it will check if the line contains exactly 2 elements separated by a tab
        if len(data_item.split("\t")) == 2:
            #append the first element(label) to y
            y.append(data_item.split("\t")[0])
            #append the second element(text) to x
            x.append(data_item.split("\t")[1])
print("success")

success


In [4]:
len(x),len(y)

(10574, 10574)

In [5]:
## prep your data 
data_dict = {"mail_message":x,"label":y}
df = pd.DataFrame(data_dict)
df.head()

ModuleNotFoundError: No module named 'pandas.io.formats.string'

ModuleNotFoundError: No module named 'pandas.io.formats.html'

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10574 entries, 0 to 10573
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   mail_message  10574 non-null  object
 1   label         10574 non-null  object
dtypes: object(2)
memory usage: 165.3+ KB


In [7]:
for i in range(0,len(df)):
#this line uses a regular expression to remove anything that isn't a letter
    message = re.sub("[^a-zA-Z]"," ",df["mail_message"][i])
    message = message.lower()
    df["mail_message"][i] = message
df.head()

C:\Users\Vishal\AppData\Local\Temp\ipykernel_12252\3555245048.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["mail_message"][i] = message


ModuleNotFoundError: No module named 'pandas.io.formats.string'

ModuleNotFoundError: No module named 'pandas.io.formats.html'

In [8]:
df["label"].value_counts()

label
spam    5747
ham     4827
Name: count, dtype: int64

In [9]:
##filter 
ham = df[df["label"] == "ham"]
spam = df[df["label"] == "spam"].sample(n=4827)

df = pd.concat([ham,spam],axis=0)
df["label"].value_counts()

label
ham     4827
spam    4827
Name: count, dtype: int64

In [10]:
df["label"] = df["label"].map({"ham":0,"spam":1})
df.head()


ModuleNotFoundError: No module named 'pandas.io.formats.string'

,mail_message,label
5000,go until jurong point crazy available only ...,0
5001,ok lar joking wif u oni,0
5003,u dun say so early hor u c already then say,0
5004,nah i don t think he goes to usf he lives aro...,0
5006,even my brother is not like to speak with me ...,0


In [11]:
df.isnull().sum()
df.duplicated().sum()


np.int64(3704)

In [12]:
## x and y 
x = df["mail_message"]
y = df["label"]

In [13]:
##prepare the single list of mails 
x = x.values

In [14]:
x[0:5]

array(['go until jurong point  crazy   available only in bugis n great world la e buffet    cine there got amore wat   ',
       'ok lar    joking wif u oni   ',
       'u dun say so early hor    u c already then say   ',
       'nah i don t think he goes to usf  he lives around here though',
       'even my brother is not like to speak with me  they treat me like aids patent '],
      dtype=object)

In [15]:
type(x)

numpy.ndarray

In [16]:
##Bag of words 
Bow_obj = CountVectorizer(
    max_features=2000,
    stop_words="english"
)
##to array -- sparse matrix -- dense matrix
x_transformed  = Bow_obj.fit_transform(x).toarray()
x_transformed.shape

(9654, 2000)

In [17]:
joblib.dump(Bow_obj,"vectorizer.lb")


['vectorizer.lb']

In [18]:
#train test split 
x_train , x_test , y_train, y_test = train_test_split(
    x_transformed ,y , 
    test_size=0.2 
)

In [19]:
BNB = BernoulliNB()
BNB.fit(x_train,y_train)


,alpha,1.0
,force_alpha,True
,binarize,0.0
,fit_prior,True
,class_prior,None


In [20]:
BNB.score(x_test,y_test)
BNB.score(x_train,y_train)

0.983167163019552

In [21]:
pred = BNB.predict(x_test)
pred

array([0, 1, 1, ..., 0, 0, 1], shape=(1931,))

In [22]:
comapre_dict = {"actual":y_test,"pred" : pred}
comapre_df = pd.DataFrame(comapre_dict)
comapre_df.head()

ModuleNotFoundError: No module named 'pandas.io.formats.string'

,actual,pred
8746,0,0
2276,1,1
3445,1,1
2396,1,1
5328,0,0


In [23]:
pickle.dump(BNB,open("spamclassifiermodel.pkl","wb"))